# Análise de Dados das Eleições 

Esse notebook tem como objetivo fazer um análise e construir visualizações da dos relacionados às eleições. Para tal, serão utilizados os dados das eleições a partir do site [Base dos Dados](https://basedosdados.org). 

## Imports necessários 

Estão sendo utilizados os seguintes pacotes: 

* `basedosdados`: API do projeto Base dos Dados para acesso as base de dados e realizar consultas. 
* `pandas`: Biblioteca para manipulação das bases com python.

In [1]:
import pandas as pd
import basedosdados as bd

## Sobre a análise

O objetivo é usar os dados das eleições para traçar o perfil dos candidados ao longo das últimas eleições. Para isso, serão utilizadas as seguintes tabelas:

* `basedosdados.br_tse_eleicoes.candidados`: para extrair a lista de candidados das últimas eleições. 
* `basedosdados.br_tse_eleicoes.resultados_candidatos`: para extrair informações como quantidade de votos e resultado. 

Para restringir os escopo de nossa análise vamos focar em algunas aspectos: 

* Serão analisados os dados somente do estado de Sergipe. No entanto, o código pode ser facilmente modificado para gerar análises de outros estados. 
* Vamos focar nos dados dos vereadores por ter um maior número de candidados e permitir uma melhor análise quando agruparmos esses dados. 
* As análises vão focar em enteder a distribuição dos candidados de acordo com o gênero.
* Serão feitos alguns recortes para visualizar aspectos de raça, aspectos espcíficos da capital e de alguns municípios. 


Vamos começar :) 

## Funções Auxiliares

In [106]:
def calculate_total_percent(data):
    for index, row in data.iterrows():
        sum_ = data.loc[index[0], 'id_candidato_bd'].sum()
        data.loc[index[0], 'total'] = int(sum_)

    data['porcentagem'] = data['id_candidato_bd']/data['total']
    data.reset_index(inplace=True)
    data.rename(columns={'id_candidato_bd': 'total_por_genero'},inplace=True)
    return data
    

## Análise para o cargo de vereador

### Lista de Candidatos

Para a lista de candidatos foi realizada uma consulta unindo as tabelas de candidatos e resultados_candidatos filtrando por UF (SE) e CARGO (Vereador). O nome do município é retornado a partir da tabela de diretórios.

```sql 
SELECT 
    b1.ano, b2.sigla_uf, b2.id_municipio, b3.nome as nome_municipio, b1.tipo_eleicao, b1.id_candidato_bd, b1.nome_urna, b1.sigla_partido, 
    b1.cargo, b1.situacao, b2.resultado, b2.votos, b1.genero, b1.raca, b1.idade
FROM 
  `basedosdados.br_tse_eleicoes.candidatos` b1,
  `basedosdados.br_tse_eleicoes.resultados_candidato` b2
INNER JOIN `basedosdados.br_bd_diretorios_brasil.municipio` b3
ON b3.id_municipio = b1.id_municipio
WHERE 
  b1.id_candidato_bd = b2.id_candidato_bd and b1.ano = b2.ano and b2.sigla_uf = 'SE' and b2.cargo = 'vereador'
ORDER BY b1.ano, b1.id_candidato_bd

```

In [114]:
query_candidatos = """
SELECT 
    b1.ano, b2.sigla_uf, b2.id_municipio, b3.nome as nome_municipio, b1.tipo_eleicao, b1.id_candidato_bd, b1.nome_urna, b1.sigla_partido, 
    b1.cargo, b1.situacao, b2.resultado, b2.votos, b1.genero, b1.raca, b1.idade
FROM 
  `basedosdados.br_tse_eleicoes.candidatos` b1,
  `basedosdados.br_tse_eleicoes.resultados_candidato` b2
INNER JOIN `basedosdados.br_bd_diretorios_brasil.municipio` b3
ON b3.id_municipio = b1.id_municipio
WHERE 
  b1.id_candidato_bd = b2.id_candidato_bd and b1.ano = b2.ano and b2.sigla_uf = 'SE' and b2.cargo = 'vereador'
ORDER BY b1.ano, b1.id_candidato_bd;
"""

In [115]:
df_candidatos = bd.read_sql(query=query_candidatos,billing_project_id='adolfo-dev')

Downloading: 100%|██████████| 28308/28308 [00:08<00:00, 3341.15rows/s]


In [116]:
df_candidatos

,ano,sigla_uf,id_municipio,nome_municipio,tipo_eleicao,id_candidato_bd,nome_urna,sigla_partido,cargo,situacao,resultado,votos,genero,raca,idade
0,2000,SE,2806305,Santa Luzia do Itanhy,eleicao ordinaria,1000073,João Donato Filho,PSDB,vereador,deferido,suplente,28,masculino,None,36.0
1,2000,SE,2804805,Nossa Senhora do Socorro,eleicao ordinaria,1002012,Galego Pedreiro,PGT,vereador,deferido,nao eleito,27,masculino,None,36.0
2,2000,SE,2800308,Aracaju,eleicao ordinaria,1002075,Ivanildo Da Silva,PTB,vereador,deferido,suplente,308,masculino,None,36.0
3,2000,SE,2807501,Tomar do Geru,eleicao ordinaria,1002761,Vardo,PDT,vereador,deferido,suplente,71,masculino,None,54.0
4,2000,SE,2805604,Porto da Folha,eleicao ordinaria,1003666,Pedro Rodrigues De Souza,PSL,vereador,deferido,suplente,136,masculino,None,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28303,2020,SE,2803203,Itaporanga d'Ajuda,eleicao ordinaria,99864,Sandro Do Sapé,CIDADANIA,vereador,deferido,nao eleito,26,masculino,parda,37.0
28304,2020,SE,2801405,Carira,eleicao ordinaria,9991,José Calazans,PSB,vereador,deferido,nao eleito,101,masculino,parda,42.0
28305,2020,SE,2801702,Cristinápolis,eleicao ordinaria,999139,Bambam Do Povão,PT,vereador,deferido,suplente,115,feminino,parda,37.0
28306,2020,SE,2803500,Lagarto,eleicao ordinaria,999390,Telma Do Santo Antonio,PSD,vereador,deferido,suplente,34,feminino,parda,63.0


### Agrupamento por Ano e Gênero

O primeiro passo foi agrupar as informações de todos os candidatos por ano e gênero para obter a proporção de candidaturas por gênero em cada ano do pleito eleitoral. A função `calculate_total_percent`, processa o resultado do agrupamento para obter as porcentagens por gênero. 

In [117]:
df_total_vereadores = df_candidatos[['id_candidato_bd','ano','genero']].groupby(by=['ano','genero']).count()
df_total_vereadores = calculate_total_percent(df_total_vereadores)
df_total_vereadores

,ano,genero,total_por_genero,total,porcentagem
0,2000,feminino,853,4372.0,0.195105
1,2000,masculino,3519,4372.0,0.804895
2,2004,feminino,943,4050.0,0.232840
3,2004,masculino,3107,4050.0,0.767160
4,2008,feminino,816,3700.0,0.220541
5,2008,masculino,2884,3700.0,0.779459
6,2012,feminino,1582,5000.0,0.316400
7,2012,masculino,3418,5000.0,0.683600
8,2016,feminino,1680,5106.0,0.329025
9,2016,masculino,3426,5106.0,0.670975


In [118]:
df_total_vereadores.to_csv("../output/total_vereadores_por_ano_genero.csv", index=False)

### Agrupamento por ano e gênero dos vereadores eleitos

Uma tabela semelhante a anterior foi gerada as pessoas eleitas. Para selecionar os vereadores eleitos, os dados foram filtrados pelos resultados: `['eleito por qp','eleito','eleito por media']`.

In [119]:
df_query_vereadores_eleitos = df_candidatos.query("resultado in ['eleito por qp','eleito','eleito por media']", engine='python')
df_vereadores_eleitos = df_query_vereadores_eleitos[['id_candidato_bd','ano','genero']].groupby(by=['ano','genero']).count()
df_vereadores_eleitos = calculate_total_percent(df_vereadores_eleitos)
df_vereadores_eleitos

,ano,genero,total_por_genero,total,porcentagem
0,2000,feminino,136,797.0,0.170640
1,2000,masculino,661,797.0,0.829360
2,2004,feminino,115,693.0,0.165945
3,2004,masculino,578,693.0,0.834055
4,2008,feminino,103,695.0,0.148201
5,2008,masculino,592,695.0,0.851799
6,2012,feminino,122,795.0,0.153459
7,2012,masculino,673,795.0,0.846541
8,2016,feminino,129,801.0,0.161049
9,2016,masculino,672,801.0,0.838951


In [120]:
df_vereadores_eleitos.to_csv("../output/total_vereadores_eleitos_por_ano_genero.csv", index=False)

### Recorte de Cor/Raça

A proposta agora é fazer um recorte dessas proporções de acordo com a cor e a raça. Como o objetivo é olhar somente dentro das pessoas de gênero feminino, os dados foram filtrados. 

In [121]:
df_total_vereadores_raca = df_candidatos.query("genero == 'feminino'")
df_total_vereadores_raca = df_total_vereadores_raca[['id_candidato_bd','ano','genero','raca']].groupby(by=['ano','genero','raca']).count()
df_total_vereadores_raca = calculate_total_percent(df_total_vereadores_raca)
df_total_vereadores_raca

,ano,genero,raca,total_por_genero,total,porcentagem
0,2016,feminino,amarela,14,1680.0,0.008333
1,2016,feminino,branca,385,1680.0,0.229167
2,2016,feminino,indigena,4,1680.0,0.002381
3,2016,feminino,parda,1131,1680.0,0.673214
4,2016,feminino,preta,146,1680.0,0.086905
5,2020,feminino,amarela,12,2063.0,0.005817
6,2020,feminino,branca,442,2063.0,0.214251
7,2020,feminino,indigena,3,2063.0,0.001454
8,2020,feminino,parda,1338,2063.0,0.648570
9,2020,feminino,preta,268,2063.0,0.129908


In [122]:
df_total_vereadores_raca.to_csv("../output/total_vereadores_por_ano_genero_raca.csv", index=False)

O mesmo recorte foi feito só para as pessoas eleitas. 

In [123]:
df_query_vereadores_eleitos_raca = df_candidatos.query("genero == 'feminino' and resultado in ['eleito por qp','eleito','eleito por media']", engine='python')
df_vereadores_eleitos_raca = df_query_vereadores_eleitos_raca[['id_candidato_bd','ano','genero','raca']].groupby(by=['ano','genero','raca']).count()
df_vereadores_eleitos_raca = calculate_total_percent(df_vereadores_eleitos_raca)
df_vereadores_eleitos_raca

,ano,genero,raca,total_por_genero,total,porcentagem
0,2016,feminino,branca,42,129.0,0.325581
1,2016,feminino,parda,84,129.0,0.651163
2,2016,feminino,preta,3,129.0,0.023256
3,2020,feminino,amarela,3,131.0,0.022901
4,2020,feminino,branca,35,131.0,0.267176
5,2020,feminino,indigena,1,131.0,0.007634
6,2020,feminino,parda,83,131.0,0.633588
7,2020,feminino,preta,9,131.0,0.068702


In [124]:
df_vereadores_eleitos_raca.to_csv("../output/total_vereadores_eleitos_por_ano_genero_raca.csv", index=False)

### Recorte de Aracaju

Com o objetivo de olhar mais especificamente para o município de Aracaju (capital do estado), foi filtrado todos os candidatos do muicípio que foram eleitos. 

In [125]:
df_vereadores_eleitos_aracaju = df_query_vereadores_eleitos.query("id_municipio == '2800308'")
df_vereadores_eleitos_aracaju.sort_values(by=['ano','votos'], inplace=True, ascending=[True, False])
df_vereadores_eleitos_aracaju

,ano,sigla_uf,id_municipio,nome_municipio,tipo_eleicao,id_candidato_bd,nome_urna,sigla_partido,cargo,situacao,resultado,votos,genero,raca,idade
3785,2000,SE,2800308,Aracaju,eleicao ordinaria,829690,Barreto,PPS,vereador,deferido,eleito,4947,masculino,None,36.0
3271,2000,SE,2800308,Aracaju,eleicao ordinaria,722885,Chiquinho,PT,vereador,deferido,eleito,4729,masculino,None,44.0
3297,2000,SE,2800308,Aracaju,eleicao ordinaria,723744,Sérgio Carlos,PSDB,vereador,deferido,eleito,4046,masculino,None,42.0
2931,2000,SE,2800308,Aracaju,eleicao ordinaria,658924,Bomfim,PPS,vereador,deferido,eleito por media,3300,masculino,None,56.0
2827,2000,SE,2800308,Aracaju,eleicao ordinaria,626985,Carlos Magal,PT,vereador,deferido,eleito,3168,masculino,None,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27544,2020,SE,2800308,Aracaju,eleicao ordinaria,694328,Professora Ângela Melo,PT,vereador,deferido,eleito por qp,1882,feminino,amarela,64.0
24410,2020,SE,2800308,Aracaju,eleicao ordinaria,1591,Sgt. Byron Estrelas Do Mar,REPUBLICANOS,vereador,deferido,eleito por media,1743,masculino,parda,42.0
27736,2020,SE,2800308,Aracaju,eleicao ordinaria,774078,Paquito De Todos,SOLIDARIEDADE,vereador,deferido,eleito por media,1486,masculino,branca,59.0
22362,2020,SE,2800308,Aracaju,eleicao ordinaria,1038140,Cícero Do Santa Maria,PODE,vereador,deferido,eleito por media,1470,masculino,branca,57.0


In [126]:
df_vereadores_eleitos_aracaju.to_csv("../output/lista_vereadores_eleitos_aracaju_por_ano.csv", index=False)

### Recorte por Cidade

Por fim, outra seleção necessária foi a de verificar a distribuição de gênero de acordo com os municípios. Da lista geral de candidatos, foram gerados os dados somente dos eleitos. 

In [127]:
df_vereadores_eleitos = df_candidatos.query("resultado in ['eleito por qp','eleito','eleito por media']", engine='python')
df_vereadores_eleitos

,ano,sigla_uf,id_municipio,nome_municipio,tipo_eleicao,id_candidato_bd,nome_urna,sigla_partido,cargo,situacao,resultado,votos,genero,raca,idade
6,2000,SE,2807006,São Miguel do Aleixo,eleicao ordinaria,1003908,Mário De Cosme,PSDB,vereador,deferido,eleito,93,masculino,None,36.0
13,2000,SE,2806701,São Cristóvão,eleicao ordinaria,1006669,Nene,PSDB,vereador,deferido,eleito por media,569,masculino,None,35.0
19,2000,SE,2807303,Telha,eleicao ordinaria,1008293,Bonifácio,PPS,vereador,deferido,eleito,94,masculino,None,36.0
31,2000,SE,2800704,Brejo Grande,eleicao ordinaria,1012597,Vaguinho,PPS,vereador,deferido,eleito,252,masculino,None,33.0
33,2000,SE,2801108,Canhoba,eleicao ordinaria,1012786,Alberto Guimaraes,PFL,vereador,deferido,eleito,152,masculino,None,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28264,2020,SE,2804300,Muribeca,eleicao ordinaria,974,Fabiano Do Camara,PL,vereador,deferido,eleito por qp,383,masculino,parda,41.0
28268,2020,SE,2806305,Santa Luzia do Itanhy,eleicao ordinaria,975006,Professor Jorge,REPUBLICANOS,vereador,deferido,eleito por qp,276,masculino,parda,56.0
28283,2020,SE,2804102,Moita Bonita,eleicao ordinaria,983959,Lêda Costa,PL,vereador,deferido,eleito por qp,288,feminino,parda,76.0
28290,2020,SE,2803302,Japaratuba,eleicao ordinaria,98758,Geovania Do Travessão,PL,vereador,deferido,eleito por qp,416,feminino,preta,37.0


Como o objetivo da análise é somente olhar para as proporções de gênro, esses dados foram agrupados de acordo com o ano, município e gênero. 

In [128]:
df_vereadores_eleitos_municipios = df_vereadores_eleitos[['ano','id_municipio','nome_municipio','genero','id_candidato_bd']].groupby(by=['ano','id_municipio','nome_municipio','genero']).count()
df_vereadores_eleitos_municipios

id_candidato_bd
ano  id_municipio nome_municipio          genero                    
2000 2800100      Amparo de São Francisco feminino                 3
                                          masculino                6
     2800209      Aquidabã                feminino                 1
                                          masculino               10
     2800308      Aracaju                 feminino                 3
...                                                              ...
2020 2807402      Tobias Barreto          masculino               12
     2807501      Tomar do Geru           feminino                 3
                                          masculino                6
     2807600      Umbaúba                 feminino                 1
                                          masculino               10

[821 rows x 1 columns]

In [129]:
for index, row in df_vereadores_eleitos_municipios.iterrows():
    sum_ = df_vereadores_eleitos_municipios.loc[(index[0], index[1]), 'id_candidato_bd'].sum()
    df_vereadores_eleitos_municipios.loc[(index[0], index[1]), 'total'] = int(sum_)

df_vereadores_eleitos_municipios['porcentagem'] = df_vereadores_eleitos_municipios['id_candidato_bd']/df_vereadores_eleitos_municipios['total']
df_vereadores_eleitos_municipios.reset_index(inplace=True)
df_vereadores_eleitos_municipios.rename(columns={'id_candidato_bd': 'total_por_genero'},inplace=True)
df_vereadores_eleitos_municipios
    

,ano,id_municipio,nome_municipio,genero,total_por_genero,total,porcentagem
0,2000,2800100,Amparo de São Francisco,feminino,3,9.0,0.333333
1,2000,2800100,Amparo de São Francisco,masculino,6,9.0,0.666667
2,2000,2800209,Aquidabã,feminino,1,11.0,0.090909
3,2000,2800209,Aquidabã,masculino,10,11.0,0.909091
4,2000,2800308,Aracaju,feminino,3,21.0,0.142857
...,...,...,...,...,...,...,...
816,2020,2807402,Tobias Barreto,masculino,12,14.0,0.857143
817,2020,2807501,Tomar do Geru,feminino,3,9.0,0.333333
818,2020,2807501,Tomar do Geru,masculino,6,9.0,0.666667
819,2020,2807600,Umbaúba,feminino,1,11.0,0.090909


In [2]:
df_vereadores_eleitos_municipios.to_csv("../output/total_vereadores_por_ano_municipio_genero.csv", index=False)

NameError: name 'df_vereadores_eleitos_municipios' is not defined

As visualizações auxiliares e outras analises podem ser vistas no documento **[VisualizacaoDosDados.ipynb](VisualizacaoDosDados.ipynb)**.